## Pandas-对齐运算

内容介绍:pandas数据类型的基础运算方法

|序号|方法|说明|
|----|----:|----:|
|01|add/radd|加法（+）,radd代表可以反转参数，即倒数|
|02|sub/rsub|减法（-）|
|03|div/rdiv|除法（/）|
|04|floordiv/rfloordiv|整除（//）|
|05|mul/rmul|乘法（\*）|
|06|pow/rpow|幂次方(\*\*)|

In [1]:
import numpy as np
import pandas as pd

In [2]:
# 示例数据
s0 = pd.Series(range(4),index=['a','b','c','d'])
print(s0)
s1 = pd.Series(range(5),index=['a','f','g','d','e'])
print(s1)
df0 = pd.DataFrame(np.arange(9).reshape(3,3),index=['a','b','c'],columns=['A','D','C'])
print(df0)
df1 = pd.DataFrame(np.arange(12).reshape(4,3),index=['a','b','f','d'],columns=['A','B','C'])
df1

a    0
b    1
c    2
d    3
dtype: int64
a    0
f    1
g    2
d    3
e    4
dtype: int64
   A  D  C
a  0  1  2
b  3  4  5
c  6  7  8


,A,B,C
a,0,1,2
b,3,4,5
f,6,7,8
d,9,10,11


### 1. 算数运算与数据对齐

普通运算会使相同的索引标签运算，不同标签会出现缺失值。<br>
运算不改变原有的数据

In [6]:
# 针对Series进行的运算
# 对齐运算：两个series进行运算的时候，相同标签的数据可以运算；不同标签的数据运算会形成缺失值。
s0+s1

a    0.0
b    NaN
c    NaN
d    6.0
e    NaN
f    NaN
g    NaN
dtype: float64

In [12]:
# 针对DataFrame进行的对齐运算
# 相同标签的数据可以运算；不同标签的数据运算会形成缺失值。
df0+df1

,A,B,C,D
a,0.0,NaN,4.0,NaN
b,6.0,NaN,10.0,NaN
c,NaN,NaN,NaN,NaN
d,NaN,NaN,NaN,NaN
f,NaN,NaN,NaN,NaN


### 2. 使用填充值的算数方法

可以解决不对应的标签出现缺失值的情况<br>
运算不改变原有的数据

In [14]:
# 默认情况下，与+相同。
s1.add(s0)

a    0.0
b    NaN
c    NaN
d    6.0
e    NaN
f    NaN
g    NaN
dtype: float64

In [30]:
# 修改fill_value参数，可以改变缺失值的设置
# 当参数设置为0时，源数据在相加的对应数据找不到对应的标签，那么元数据被设置为0
s1.add(s0,fill_value=0)

a    0.0
b    1.0
c    2.0
d    6.0
e    4.0
f    1.0
g    2.0
dtype: float64

In [20]:
#DataFrame使用fill_value参数时，当一方df有数另一方没数时，没数的一方会被填充为fill_value的值。
#当两方都没有数时，那么两方不会被填充，运算结果为NaN
df0.add(df1,fill_value = 0)

,A,B,C,D
a,0.0,1.0,4.0,1.0
b,6.0,4.0,10.0,4.0
c,6.0,NaN,8.0,7.0
d,9.0,10.0,11.0,NaN
f,6.0,7.0,8.0,NaN


In [24]:
# rdiv
print(1/df0)
print('*'*20)
df0.rdiv(1)

          A         D      C
a       inf  1.000000  0.500
b  0.333333  0.250000  0.200
c  0.166667  0.142857  0.125
********************


,A,D,C
a,inf,1.000000,0.500
b,0.333333,0.250000,0.200
c,0.166667,0.142857,0.125


In [27]:
# 先统一列标签之后再运算
df1.reindex(columns=df0.columns,fill_value=9)

,A,D,C
a,0,9,2
b,3,9,5
f,6,9,8
d,9,9,11


In [29]:
df0

,A,D,C
a,0,1,2
b,3,4,5
c,6,7,8


### 3. DataFrame和Series混合运算

一维和多维数据运算，存在广播特性<br>
这一点同numpy的广播运算相同<br>
Series的索引，匹配的是DataFrame的列索引

In [33]:
# 由于没有匹配到相应的索引，运算会出现缺失值。但包含两者行列集合(当然Series没有行的概念)。
# Series的索引，匹配pandas的列索引即可。
df0-s0

,A,C,D,a,b,c,d
a,NaN,NaN,NaN,NaN,NaN,NaN,NaN
b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
c,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
s2 = pd.Series(np.arange(2,5),index=['A','B','F'])
s2

A    2
B    3
F    4
dtype: int32

In [6]:
# 由于两者只匹配到列'A',因此运算只显示A
df0-s2

,A,B,C,D,F
a,-2.0,NaN,NaN,NaN,NaN
b,1.0,NaN,NaN,NaN,NaN
c,4.0,NaN,NaN,NaN,NaN


In [38]:
#两种数据格式不同的情况，不能使用fill_value参数
df0.sub(s2,fill_value=0)

NotImplementedError: fill_value 0 not supported.

In [7]:
# 使用运算函数，并指定axis轴为0(index)，则Series匹配各行的数据，沿着行广播运算
df0.sub(s0,axis=0)  # axis = 'index'

,A,D,C
a,0.0,1.0,2.0
b,2.0,3.0,4.0
c,4.0,5.0,6.0
d,NaN,NaN,NaN
